## Building Environments in Aviary

In this tutorial we're going to focus on the GSM8K environment [1]

In [1]:
from aviary.env import Environment, Frame, TaskDataset


In [50]:
from pydantic import BaseModel, ConfigDict, Field, field_validator 
from aviary.gsm8k.env import CalculatorEnvConfig


class CalculatorEnvConfig(BaseModel):
    model_config = ConfigDict(extra="forbid", arbitrary_types_allowed=True)

    correct_reward: float = 1.0
    incorrect_reward: float = 0.0
    tool_failure_reward: float = -1.0
    tool_success_reward: float = 0.0
    rel_tol: float = 1e-4

    done_on_failure: bool = True

In [20]:
from typing import TYPE_CHECKING, Literal
import contextlib



from aviary.message import Message
from aviary.tools import Tool, ToolRequestMessage, ToolResponseMessage

class CalculatorEnv(Environment[None]):
    def __init__(
        self,
        problem_id: str,
        problem: str,
        answer: float,
        config: CalculatorEnvConfig | None = None,
    ):
        # The problem is not part of the state because it is always the same.
        # Putting it in the state would imply it is somehow affected by .step()
        # or re-initialized by .reset().
        self.problem_id = problem_id
        self.problem = problem
        self.answer = float(answer)

        self.config = config if config is not None else CalculatorEnvConfig()

        self.calc_tool = Tool.from_function(self.calculator)
        self.check_tool = Tool.from_function(self.check_answer)
        self.tools = [self.calc_tool, self.check_tool]

    async def reset(self) -> tuple[list[Message], list[Tool]]:
        self.state = None  # this environment is effectively stateless
        return [Message(content=self.problem)], self.tools

    async def step(
        self, action: ToolRequestMessage
    ) -> tuple[list[Message], float, bool, bool]:
        if not action.tool_calls:
            return (
                [
                    Message(
                        content="Must call one of the provided tools (calculator or check_answer)."
                    )
                ],
                self.config.tool_failure_reward,
                self.config.done_on_failure,
                False,
            )

        valid_action, invalid_action = self.filter_invalid_tool_calls(action)

        invalid_response_msgs = [
            ToolResponseMessage.from_call(tool_call, content="")
            for tool_call in invalid_action.tool_calls
        ]

        if valid_action.tool_calls:
            results = await self.exec_tool_calls(valid_action)
            response_msgs = []
            total_reward = 0.0
            any_done = False

            for tool_call, result in zip(valid_action.tool_calls, results, strict=True):
                response, reward, done = json.loads(result.content)

                response_msgs.append(
                    ToolResponseMessage.from_call(tool_call, content=str(response))
                )

                total_reward += reward
                any_done |= done

            return response_msgs + invalid_response_msgs, total_reward, any_done, False

        return (
            invalid_response_msgs,
            self.config.tool_failure_reward * len(invalid_response_msgs),
            self.config.done_on_failure,
            False,
        )

    def check_answer(self, answer: str) -> tuple[bool, float, Literal[True]]:
        """Check if the proposed answer is correct.

        Args:
            answer: Proposed answer.

        Returns:
            Three-tuple of if correct, associated reward (correct_reward if correct,
                tool_failure_reward if tool failure, otherwise incorrect_reward), and
                True indicating done.
        """
        try:
            correct: bool = (
                abs(float(answer) - self.answer)
                / (abs(self.answer) + self.config.rel_tol)
                < self.config.rel_tol
            )
            reward = (
                self.config.correct_reward if correct else self.config.incorrect_reward
            )
        except ValueError:
            return False, self.config.tool_failure_reward, True
        else:
            return correct, reward, True

    def calculator(self, expr: str) -> tuple[float | str | None, float, bool]:
        """Calculate a mathematical expression.

        Args:
            expr (str): A valid python expression

        Returns:
            float: Result of the expression
        """
        try:
            expr = expr.strip()
            result = eval(expr)
            with contextlib.suppress(ValueError):
                if int(result) == result:
                    result = int(result)

        except Exception:
            return (
                "Error using calculator",
                self.config.tool_failure_reward,
                self.config.done_on_failure,
            )
        return result, self.config.tool_success_reward, False


In [27]:
from enum import StrEnum
import datasets


# SEE: https://huggingface.co/datasets/openai/gsm8k
GSM8K_PUBLIC_SOURCE = "openai/gsm8k"


class GSM8kDataset(TaskDataset):
    class Split(StrEnum):
        train_full = "train_full"  #  full training set from OpenAI
        train = "train"  # 80% of train_full (idx%5 != 0)
        val = "val"  # 20% of train_full (idx%5 == 0)
        test = "test"

    def __init__(
        self,
        split: Split | str,
        config: CalculatorEnvConfig | dict | None = None,
        hf_source: str = GSM8K_PUBLIC_SOURCE,
    ):
        if isinstance(config, dict):  # Serialized config
            config = CalculatorEnvConfig(**config)
        elif config is None:
            config = CalculatorEnvConfig()
        self.config = config

        if isinstance(split, str):
            split = self.Split(split)

        src_df = self._get_df_from_hf(hf_source, split)

        # Assign problem ID for the env
        src_df["problem_id"] = split.value + "_" + src_df.index.astype(str)

        # attempt to extract a numerical answer
        try:
            src_df["answer_num"] = src_df["answer"].apply(
                # answer is formatted as: <some text>\n#### <answer_num>
                lambda a: float(a.split("#### ")[1].replace(",", ""))
            )
        except Exception as e:
            raise RuntimeError(
                "Failed to extract numerical answer from 'answer' column"
            ) from e

        self.src_df = src_df

    def _get_df_from_hf(self, hf_source: str, split: Split) -> "pd.DataFrame":
        # All non-test splits are derived from train
        hf_split = "test" if split == self.Split.test else "train"

        kw = {}
        if hf_source == GSM8K_PUBLIC_SOURCE:
            kw["name"] = "main"  # as opposed to "socratic"

        src_df = (
            datasets.load_dataset(hf_source, split=hf_split, **kw)
            .to_pandas()
            .reset_index(drop=True)
        )
        if split == self.Split.train:
            src_df = src_df[src_df.index % 5 != 0]
        elif split == self.Split.val:
            src_df = src_df[src_df.index % 5 == 0]
        return src_df

    def get_new_env_by_idx(self, idx: int) -> CalculatorEnv:
        row = self.src_df.iloc[idx]
        return CalculatorEnv(
            problem_id=row["problem_id"],
            problem=row["question"],
            answer=row["answer_num"],
            config=self.config,
        )

    def __len__(self) -> int:
        return len(self.src_df)

In [47]:
from aviary.tools import ToolSelector, ToolSelectorLedger

dataset = GSM8kDataset(split="train")
rewards = 0

for i in range(3):
    print(f'Considering problem number {i+1}\n')
    env = dataset.get_new_env_by_idx(i)
    obs, tools = await env.reset()
    ledger = ToolSelectorLedger(tools=tools, messages=obs)
    tool_selector = ToolSelector(model_name="gpt-4o")
    for _ in range(10):
        print(f'Observation is {obs}\n')
        action = await tool_selector(ledger.messages, tools)
        print(f'Action is {action}\n')
        obs, reward, done, _ = await env.step(action)
        ledger.messages.extend([action, *obs])
        print(reward)
        rewards += reward
        if done:
            break
print(f'Total reward is {rewards}')

Considering problem number 0

Observation is [Message(role='user', content='Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?')]

Action is Tool request message '' for tool calls: calculator(expr='12 * (50/60)') [id=call_OSMkKNC9e3SFQGNVkADpgZQ2]

0.0
Observation is [ToolResponseMessage(role='tool', content='10', name='calculator', tool_call_id='call_OSMkKNC9e3SFQGNVkADpgZQ2')]

Action is Tool request message '' for tool calls: check_answer(answer='10') [id=call_83R8O3GEnDrKE2y1LNnwEeit]

1.0
Considering problem number 1

Observation is [Message(role='user', content='Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?')]

Action is Tool request message '' for tool calls: calculator(expr='15 * 2') [id=call_0WpHfGn4GV9k

## References

[1] Cobbe, K., Kosaraju, V., Bavarian, M., Chen, M., Jun, H., Kaiser, L., Plappert, M., Tworek, J., Hilton, J., Nakano, R. and Hesse, C., 2021. [Training verifiers to solve math word problems](https://arxiv.org/abs/2110.14168). arXiv preprint arXiv:2110.14168.